In [1]:
from pathlib import Path
import os

yr = 2023
d = 22

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
import numpy as np

def format_input(inp):
  formatted_input = []
  for l in inp.splitlines():
    l0 = l.split('~')[0]
    l1 = l.split('~')[1]
    formatted_input.append([[int(x) for x in l0.split(',')],
                          [int(x) for x in l1.split(',')]])
  return np.asarray(formatted_input)

In [3]:
def build_snapshot_arr(formatted_input):
  '''
  Build a 3D Numpy array with 0's in all empty spaces
  Spaces with blocks in them will contain a number
  corresponding to the index of the block which is
  occupying that space
  '''

  maxes = [np.max(formatted_input[:,:,i].flatten()) for i in range(3)] + np.array([1,1,1])
  arr = np.zeros(maxes)

  for i, block in enumerate(formatted_input):
    arr[block[0][0]:block[1][0]+1,
        block[0][1]:block[1][1]+1,
        block[0][2]:block[1][2]+1] = i+1
  return arr

In [4]:
def build_dropped_arr(formatted_input):
  '''
  Take the snapshot array and transform it
  to simulate all the blocks falling downward
  '''
  def find_lowest_possible_landing_point(block, arr, dropped_block_locs):
    '''
    If we find a block, look for the lowest possible position that
    it can fall to
    '''

    assert(np.all(arr[block[0][0]:block[1][0]+1,
        block[0][1]:block[1][1]+1,
        block[0][2]:block[1][2]+1]!=0))


    assert(len(set((arr[block[0][0]:block[1][0]+1,
        block[0][1]:block[1][1]+1,
        block[0][2]:block[1][2]+1].flatten())))==1)

    # The block id will be denoted by the value of the array
    # where the block resides
    block_n = arr[block[0][0], block[0][1], block[0][2]]

    block_height = block[1][2] - block[0][2]


    found = False
    # i represents the index that the bottom
    # of the block will eventuall end up at
    for i in reversed(range(0, block[0][2])):
      if not found:
        if(
          # If the block were to drop down to
          # the i'th z location, then the block
          # will not be occupying the same space
          # as another block
          # (it is okay if the potential location
          # overlaps with the current block location)
          np.all(
            np.isin(arr[block[0][0]:block[1][0]+1,
                      block[0][1]:block[1][1]+1,
                      i:i+block_height+1], [0, block_n])
          )
          # Either dropping the block to the i'th z location
          # will put it on the floor, or there will be another
          # block directly under it to support it
          and (i==1

               or

               np.any(arr[block[0][0]:block[1][0]+1,
                        block[0][1]:block[1][1]+1,
                        i-1:i]!=0)
            )
        ):

          # We move the block in the lines below here
          found = True
            
          arr[block[0][0]:block[1][0]+1,
              block[0][1]:block[1][1]+1,
              block[0][2]:block[1][2]+1] = 0


          arr[block[0][0]:block[1][0]+1,
                      block[0][1]:block[1][1]+1,
                      i:i+block_height+1] = block_n


          new_block = [[block[0][0], block[0][1], i], [block[1][0], block[1][1], i+block_height]]

    if found:
      # We found a lower place where this block can drop to
      dropped_block_locs.append((block_n, new_block))
    else:
      # The block had to stay where it was
      dropped_block_locs.append((block_n, block))
    return arr, dropped_block_locs




  import copy

  arr = build_snapshot_arr(formatted_input)

  last = None

  dropped_block_locs = []

  formatted_input_sorted = sorted(formatted_input,
                                   key= lambda x: (x[0,2], x[1,2]))


  # We start at the lowest level, moving up in the z-axis
  # if we encounter a block during this scan, then we drop it down as
  # far as it can go
  for block in formatted_input_sorted:


    arr, dropped_block_locs = find_lowest_possible_landing_point(block,
                                                            arr,
                                                            dropped_block_locs)


  return arr, sorted(dropped_block_locs, key=lambda x: x[0])

In [5]:
def find_supporting_blocks(dropped_arr, dropped_block_locs):
  '''
  Find the blocks that each block is supporting directly after
  the drop
  '''
  brick_to_supported = {}
  for bn, bl in dropped_block_locs:
    assert(np.all(dropped_arr[bl[0][0]:bl[1][0]+1, bl[0][1]:bl[1][1]+1, bl[0][2]:bl[1][2]+1]==bn))
    bricks_above = []
    # If directly above the current block in the z-direction, we see a number that is not zero, then this block
    # is supporting the block with the label of that number
    cur_supported = set(dropped_arr[bl[0][0]:bl[1][0]+1, bl[0][1]:bl[1][1]+1, bl[1][2]+1:bl[1][2]+2].flatten()).difference({0})
    assert(bn not in cur_supported)
    brick_to_supported[int(bn)] = list([int(x) for x in cur_supported])


  return brick_to_supported


def load_bearing_blocks_from_supporting_blocks(brick_to_supported):
  '''
  Given a dictionary denoting which blocks are supported by which other blocks
  determine which blocks are load bearing

  A load bearing block indicates that either:
    - The brick supports no other blocks
    - Every block that the block supports is also supported by at least
      one other block
  '''
    
  load_bearing = []
  non_load_bearing = []

  for bn in brick_to_supported:
    if len(brick_to_supported[bn]) == 0:
      non_load_bearing.append(bn)
    else:
      supported_elsewhere = {x: x in np.concatenate([brick_to_supported[y] if y!=bn else [] for y in brick_to_supported], axis=0)
                             for x in brick_to_supported[bn]}
      if all(supported_elsewhere.values()):
        non_load_bearing.append(bn)
      else:
        load_bearing.append(bn)



  return load_bearing, non_load_bearing


def find_load_bearing_blocks(dropped_arr, dropped_block_locs):
  return load_bearing_blocks_from_supporting_blocks(find_supporting_blocks(dropped_arr, dropped_block_locs))

In [6]:
def find_load_bearing_blocks_chain_reaction_from_supporting_blocks(brick_to_supported):
  '''
  Given a dictionary denoting which blocks are supported by which other blocks
  determine how many block would fall in a chain reaction from removing each block
  '''
  from tqdm.notebook import tqdm
  import copy

  # Reverse the dictionary so the keys are the block and the
  # values are a list of blocks that the current brick is supported by
  brick_to_supported_by = {k:[] for k in brick_to_supported}
  for k, v in brick_to_supported.items():
    for it in v:
      brick_to_supported_by[it].append(k)



  load_bearing_chained = {}

  # Maintain a queue of blocks that need to be removed
  # starting with the current block we are examining
  # When we pop a block from the queue and "remove" it
  # We scan through the dictionary and update it to denote
  # that if any block was supported by the block we just removed
  # then they no longer are.
  # When the queue is empty we check how many bricks are now
  # supported by zero blocks compared to how many unsupported
  # blocks there were before the removal of any blocks
  #  
  # We do the above for each block
  for bn in tqdm(brick_to_supported):
    cur_brick_to_supported_by = copy.deepcopy(brick_to_supported_by)
    queue = [bn]
    while len(queue)!=0:
      cur = queue.pop()
      for cbtsb in cur_brick_to_supported_by:
        if cur_brick_to_supported_by[cbtsb] == [cur]:
          queue.append(cbtsb)
        cur_brick_to_supported_by[cbtsb] = [x for x in cur_brick_to_supported_by[cbtsb] if x!=cur]
    cur_diff = np.sum([len(x)==0 for x in cur_brick_to_supported_by.values()]) - np.sum([len(x)==0 for x in brick_to_supported_by.values()])
    load_bearing_chained[bn] = cur_diff
  return load_bearing_chained

In [7]:
def find_non_load_bearing_blocks_count(formatted_input):
  dropped_arr, dropped_block_locs = build_dropped_arr(formatted_input)
  load_bearing, non_load_bearing = find_load_bearing_blocks(dropped_arr, dropped_block_locs)
  return len(non_load_bearing)

In [8]:
def find_supported_blocks_chain_reaction_sum(formatted_input):
  dropped_arr, dropped_block_locs = build_dropped_arr(formatted_input)
  brick_to_supported = find_supporting_blocks(dropped_arr, dropped_block_locs)
  load_bearing_chained = find_load_bearing_blocks_chain_reaction_from_supporting_blocks(brick_to_supported)
  return np.sum(list(load_bearing_chained.values()))

In [9]:
import time

t = time.time()

formatted_input = format_input(inp)

print(find_non_load_bearing_blocks_count(formatted_input))
print(find_supported_blocks_chain_reaction_sum(formatted_input))

print("\nRUNTIME: ", time.time()-t)

517


  0%|          | 0/1491 [00:00<?, ?it/s]

61276

RUNTIME:  40.91960644721985
